In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def data(self, images, imagesSegmentation, labels):
    ytrain = torch.eye(4)[labels]  # Convert labels to one-hot encoding
    self.images = torch.tensor(images).float()
    self.imagesSegmentation = torch.tensor(imagesSegmentation).float()
    self.labels = torch.tensor(ytrain).float()


In [3]:
class Generator(nn.Module):
    def __init__(self, latent_space, labels_dim, kernel_size):
        super().__init__()

        self.fc1 = nn.Linear(latent_space + labels_dim, 1024)

    def forward(self, noise, labels):

        x = torch.cat((noise, labels), dim=1)
        x = self.fc1(x)

        return generated_images, segmentation_images

class Discriminator(nn.Module):
    def __init__(self, img_shape, kernel_size):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size, padding='same')

    def forward(self, img_input, img_seg_input):
        x = torch.cat((img_input, img_seg_input), dim=1)
        x = self.conv1(x)

        return outputs, labels_output


In [ ]:
def trainAlgorithm(self, G, D):
    criterion = nn.MSELoss()
    criterion_ce = nn.CrossEntropyLoss()
    optimizer_D = optim.RMSprop(D.parameters(), lr=self.eta, alpha=self.weight_decay)
    optimizer_G = optim.RMSprop(G.parameters(), lr=self.eta * 0.5, alpha=self.weight_decay * 0.5)

    for epoch in range(self.epochs):
        # Get batch indices
        indexs = np.random.randint(0, len(self.images), size=self.batch_size)
        real_images = self.images[indexs]
        real_img_seg = self.imagesSegmentation[indexs]
        real_labels = self.labels[indexs]
        real_tag = torch.ones(self.batch_size)

        # Generate noise and labels
        noise = torch.randn(self.batch_size, self.latent_space)
        fake_labels = torch.tensor(np.eye(4)[np.random.choice(4, size=self.batch_size)])

        # Generate fake images and segmentations
        with torch.no_grad():
            fake_images, fake_segmentations = G(noise, fake_labels)

        fake_tag = torch.zeros(self.batch_size)

        # Combine real and fake data
        all_images = torch.vstack([real_images, fake_images])
        all_segmentations = torch.vstack([real_img_seg, fake_segmentations])
        all_labels = torch.vstack([real_labels, fake_labels])
        all_tags = torch.hstack([real_tag, fake_tag])

        # Train discriminator
        D.train()
        optimizer_D.zero_grad()
        d_output, d_labels_output = D(all_images, all_segmentations)
        d_loss_tag = criterion(d_output, all_tags)
        d_loss_labels = criterion_ce(d_labels_output, all_labels.argmax(dim=1))
        d_loss = d_loss_tag + d_loss_labels
        d_loss.backward()
        optimizer_D.step()

        # Train GAN
        GAN.train()
        optimizer_GAN.zero_grad()
        gan_noise = torch.randn(self.batch_size, self.latent_space)
        gan_labels = torch.tensor(np.eye(4)[np.random.choice(4, size=self.batch_size)])
        gan_output = GAN(gan_noise, gan_labels)
        g_loss_tag = criterion(gan_output[0], real_tag)
        g_loss_labels = criterion_ce(gan_output[1], gan_labels.argmax(dim=1))
        g_loss = g_loss_tag + g_loss_labels
        g_loss.backward()
        optimizer_GAN.step()

        # Print loss periodically
        if epoch % 5000 == 0:
            print(f'Epoch: {epoch}')
            print(f'Discriminator loss: [tag: {d_loss_tag.item()}, labels: {d_loss_labels.item()}], '
                  f'Generator loss: [tag: {g_loss_tag.item()}, labels: {g_loss_labels.item()}]')
            self.samples(G, gan_noise, gan_labels)
